<div class="alert alert-block alert-warning">

# **LoadComicData_lab.ipynb**
***
### **This script loads comic book images for LAB neural network training. The images are made in a pair of one colored, and one grayscaled.**


In [14]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

### Loads API-key

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '..\API Key here'

### Download the dataset - **Warning it's 2 GB**

In [16]:
print("Downloading dataset from Kaggle...")
!kaggle datasets download -d cenkbircanoglu/comic-books-classification -p comic_dataset --unzip

Traceback (most recent call last):
  File "C:\Users\sebas\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\sebas\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\sebas\Desktop\PythonMetaRepo\CNN_U-Net_GAN_Project\.venv\Scripts\kaggle.exe\__main__.py", line 7, in <module>
  File "c:\Users\sebas\Desktop\PythonMetaRepo\CNN_U-Net_GAN_Project\.venv\lib\site-packages\kaggle\cli.py", line 68, in main
    out = args.func(**command_args)
  File "c:\Users\sebas\Desktop\PythonMetaRepo\CNN_U-Net_GAN_Project\.venv\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
  File "c:\Users\sebas\Desktop\PythonMetaRepo\CNN_U-Net_GAN_Project\.venv\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.co

### Process images into grayscale and colored pairs and splitting some proportion of the data into training data and test data. 

In [5]:
from sklearn.model_selection import train_test_split

input_images = []
output_images = []

image_paths = glob.glob('comic_dataset/**/*.jpg', recursive=True)
print(f"Found {len(image_paths)} images.")

for path in tqdm(image_paths[:2000]):
    color_img = cv2.imread(path)
    if color_img is None:
        continue
    color_img = cv2.resize(color_img, (256, 256))
    lab_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2LAB)

    l_channel = lab_img[..., 0:1] / 255.0                # Input grayscale (L)
    ab_channels = (lab_img[..., 1:] / 128.0) - 1.0       # Output color (AB), scaled to [-1, 1]

    input_images.append(l_channel)
    output_images.append(ab_channels)

X = np.array(input_images)
Y = np.array(output_images)


print(f"Input shape: {X.shape}, Output shape: {Y.shape}")

# Split to avoid testing on same data the model is trained on. 20% (0.2) goes to testing  
X_train, X_test, y_train, y_test = train_test_split(
    input_images, output_images, test_size=0.2, random_state=42)

ModuleNotFoundError: No module named 'sklearn'

### Save processed data

In [ ]:
# Creates folder if it doesn't exist yet
os.makedirs("prepared_data/LAB", exist_ok=True)

# Saves files
np.save("prepared_data/LAB/comic_input_grayscale_train.npy", X_train)
np.save("prepared_data/LAB/comic_output_color_train.npy", y_train)
np.save("prepared_data/LAB/comic_input_grayscale_test.npy", X_test)
np.save("prepared_data/LAB/comic_output_color_test.npy", y_test)
print("LAB data saved to prepared_data/LAB/")

### Display processed data to confirm stuff is working.

In [ ]:
num_samples = 10
plt.figure(figsize=(12, 4))
for i in range(num_samples):
    plt.subplot(2, num_samples, i+1)
    plt.imshow(X[i].squeeze(), cmap='gray')
    plt.title("Grayscale")
    plt.axis('off')

    plt.subplot(2, num_samples, i+1+num_samples)
    plt.imshow(Y[i])
    plt.title("Color")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Display from test 
plt.subplot(1, 2, 1)
plt.imshow(X_test[0].squeeze(), cmap='gray')
plt.title("Grayscale Input")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(y_test[0])
plt.title("Color Output")
plt.axis('off')
plt.show()

In [ ]:
# Display from train
plt.subplot(1, 2, 1)
plt.imshow(X_train[0].squeeze(), cmap='gray')
plt.title("Grayscale Input")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(y_train[0])
plt.title("Color Output")
plt.axis('off')
plt.show()
